In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import multiprocessing

In [2]:
pimagepath = os.getcwd() + "/sub/pos_sub/"
os.makedirs(pimagepath, exist_ok=True)

In [3]:
POSL=os.listdir("{}/positiveLR/".format(os.getcwd()))
 
print("Total: POS={}".format(len(POSL)))

Total: POS=32788


In [4]:
def sliding_window(image, stride, imgSize):
    height, width, _ = image.shape
    img = []
    a1 = list(range(0, height-imgSize+stride, stride))
    a2 = list(range(0, width-imgSize+stride, stride))
    if (a1[-1]+imgSize != height):
        a1[-1] = height-imgSize
    if (a2[-1]+imgSize != width):
        a2[-1] = width-imgSize
    for y in a1:
        for x in a2:
            im1 = image[y:y+imgSize, x:x+imgSize, :]
            img.append(np.array(im1))
    return img

In [5]:
def handle_one_image(path, sType):
    image_data = []
    im = Image.open(path)
    im = np.array(im)
    if (im.shape[0] >= SIZE and im.shape[1] >= SIZE):
        img = sliding_window(im, STRIDE, SIZE)
        for i in range(len(img)):
            if(img[i].shape[2] >=3):
                image_data.append(1)
        return image_data, sType
    else:
        # indicate that no images are available
        return [], sType

In [6]:
SIZE = 128
STRIDE = 64
pimage_dict = {}
nimage_dict = {}
psubimage_counter = 0
pimage_counter = 0
nsubimage_counter = 0
nimage_counter = 0

In [7]:
def gen_sub_img(posl):
    global pimage_dict
    global nimage_dict
    global psubimage_counter
    global pimage_counter
    global nsubimage_counter
    global nimage_counter
    print("Creating Sub images and generating dictionary")
    # Create a list of tuples consisting of the file path, and the class
    # dictionary info for each of the cl arguments
    args = []
    for img in posl:
        path = os.getcwd() + "/positiveLR/" + img
        args.append((path, "PS_"))
            
    num_workers = multiprocessing.cpu_count()  

    with multiprocessing.Pool(processes = num_workers) as pool:   # or however many processes
        image_counter = 0
        
        # Use multiprocessing to call handle_on_image(pathname, info)
        # and return the results in order
        for images, sType in pool.starmap(handle_one_image, args):
            image_list = []
            # Images is a list of returned images.  info is the class_dictionary info that we passed
            if(sType == "PS_"):
                for image in images:
                    image_list.append(psubimage_counter)
                    psubimage_counter += 1
                pimage_dict[pimage_counter] = image_list
                pimage_counter += 1
            else:
                for image in images:
                    image_list.append(nsubimage_counter)
                    nsubimage_counter += 1
                nimage_dict[nimage_counter] = image_list
                nimage_counter += 1
    print("Sliding window process finished")

In [8]:
data_partition = 10
divp = int(len(POSL)/data_partition)
fromidx = 0
toidx = 0
for i in range(data_partition):
    fromidx, toidx = i*divp, (i+1)*divp
    P = POSL[fromidx : toidx]
    gen_sub_img(P)
    print("Sub-image generation finished for data part -> {}".format(i+1))

Creating Sub images and generating dictionary
Sliding window process finished
Sub-image generation finished for data part -> 1
Creating Sub images and generating dictionary
Sliding window process finished
Sub-image generation finished for data part -> 2
Creating Sub images and generating dictionary
Sliding window process finished
Sub-image generation finished for data part -> 3
Creating Sub images and generating dictionary
Sliding window process finished
Sub-image generation finished for data part -> 4
Creating Sub images and generating dictionary
Sliding window process finished
Sub-image generation finished for data part -> 5
Creating Sub images and generating dictionary
Sliding window process finished
Sub-image generation finished for data part -> 6
Creating Sub images and generating dictionary
Sliding window process finished
Sub-image generation finished for data part -> 7
Creating Sub images and generating dictionary
Sliding window process finished
Sub-image generation finished for

In [9]:
len(nimage_dict.keys())

0

In [10]:
pli = [len(pimage_dict[k]) for k in pimage_dict.keys()]

In [11]:
pli = [len(pimage_dict[k]) for k in pimage_dict.keys() if len(pimage_dict[k]) != 0]

In [12]:
np.mean(pli)

40.689469696969695

In [13]:
print("Mean : positive -> {} ;".format(np.mean(pli)))
print("SD : positive -> {} ;".format(np.std(pli)))
print("MAX : positive -> {} ;".format(np.max(pli)))
print("MIN : positive -> {} ;".format(np.min(pli)))

Mean : positive -> 40.689469696969695 ;
SD : positive -> 38.84435446704196 ;
MAX : positive -> 324 ;
MIN : positive -> 1 ;


In [14]:
import json
with open("pimage_dict.json", "w") as outfile:
    json.dump(pimage_dict, outfile) 